<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/EDA_Yfinance_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACD→ Analise das ações utilizando as médias moveis como orientação para compra e venda.

In [ ]:
# Instalando Yfinance API
!pip install Yfinance

In [ ]:
# Importando as bibliotecas
import warnings
import numpy                as np
import pandas               as pd
import yfinance             as yf
import seaborn              as sns
import plotly.graph_objects as go

In [ ]:
# Help function
# Configuração do padrão de visualização dos dados e graficos
# https://seaborn.pydata.org/tutorial.html
sns.set_theme(
    context='talk',
    style='ticks',
    palette='icefire',
    font_scale=.8,
    rc={'figure.figsize': (25, 15)}
)

warnings.filterwarnings("ignore")
     

In [ ]:
# Escolhendo a ação
paper = 'PETR4.SA'

ticker = yf.Ticker(paper)

# Intervalo de tempo
df = ticker.history(period='30d')
df = df['Close']

df_ticker_daily = ticker.history(period='1d', interval='5m')

df_ticker_daily = df_ticker_daily['Close']

In [ ]:
info = ticker.info
acao = info['symbol']
acao

In [ ]:
df = pd.DataFrame(df)
df_ticker_daily = pd.DataFrame(df_ticker_daily)

In [ ]:
df_ticker_daily.reset_index(inplace=True)

# Quando aplicamos o Yfinance, ele acaba trazendo as informações do momento atual
# e neste momento não queremos isso
#df.drop(df.tail(1).index,inplace=True)
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
# Criando um novo Df com os dados historicos
df_ticker = df

In [ ]:
# Remove a ultima data do df_daily
df_remove_ticket = df_ticker.loc[(df_ticker['Date'] == pd.to_datetime('today').normalize())]
df_ticker = df_ticker.drop(df_remove_ticket.index)
df_ticker

In [ ]:
## Coletando o ultimo preço negociado até o agora
#df_ticker_daily_last_price = df_ticker_daily.tail(1)
#df_ticker_daily_last_price

In [ ]:
## Ajustar o formato da data e nome coluna
#df_ticker_daily_last_price.rename(columns={'Datetime':'Date'}, inplace=True)
#df_ticker_daily_last_price

In [ ]:
#df_ticker_daily_last_price_formated = pd.to_datetime(df_ticker_daily_last_price['Date'], format='%Y-%m-%d')

In [ ]:
# Append data atual
#df_ticker = df_ticker.append(df_ticker_daily_last_price)
df_ticker

In [ ]:
#Ajustando a data
df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], utc=True).dt.date
df_ticker

In [ ]:
# Calculando o MACD
rapidaMME=df_ticker.Close.ewm(span=12).mean()
lentaMME=df_ticker.Close.ewm(span=26).mean()

MACD = rapidaMME - lentaMME

sinal = MACD.ewm(span=9).mean()

df_ticker['MACD'] = MACD
df_ticker['sinal'] = sinal
df_ticker

In [ ]:
# Codigo para decidiar compra e venda
df_ticker.at[0, 'flag'] = ''
df_ticker['preco_compra']=np.nan
df_ticker['preco_venda']=np.nan


for i in range(1, len(df_ticker.sinal)):
  if df_ticker['MACD'].iloc[i] > df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'C':
      df_ticker['flag'][i] = 'C'
    else:
      df_ticker['flag'][i] = 'C'
      df_ticker['preco_compra'][i] = df_ticker['Close'][i]

  elif df_ticker['MACD'].iloc[i] < df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'V':
      df_ticker['flag'][i] = 'V'
    else:
      df_ticker['flag'][i] = 'V'
      df_ticker['preco_venda'][i] = df_ticker['Close'][i]

df_ticker


In [ ]:
#Visualização grafica
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [ ]:
df_plot = df_ticker

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['Close'],
                         name="Preço fechamento",
                         line_color='orange'
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_compra'],
                         name="Compra",
                         mode='markers',
                         marker=  dict(
                            color='green',
                            size=10,
                            )
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_venda'],
                         name="Venda",
                         mode='markers',
                         marker=  dict(
                            color='red',
                            size=10,
                            )
                         ))

fig.show()